In [62]:
import time

import mujoco.viewer
import mujoco
import numpy as np
import torch
import yaml


def get_gravity_orientation(quaternion):
    qw = quaternion[0]
    qx = quaternion[1]
    qy = quaternion[2]
    qz = quaternion[3]

    gravity_orientation = np.zeros(3)

    gravity_orientation[0] = 2 * (-qz * qx + qw * qy)
    gravity_orientation[1] = -2 * (qz * qy + qw * qx)
    gravity_orientation[2] = 1 - 2 * (qw * qw + qz * qz)

    return gravity_orientation


def pd_control(target_q, q, kp, target_dq, dq, kd):
    """Calculates torques from position commands"""
    return (target_q - q) * kp + (target_dq - dq) * kd


In [63]:
# Load configuration
import os

# For Jupyter notebook, use current working directory
mujoco_dir = os.getcwd()
h12_root = os.path.dirname(mujoco_dir)

config_file = "h12.yaml"
config_path = os.path.join(mujoco_dir, config_file)

print(f"Loading config from: {config_path}")
print(f"H12 root: {h12_root}")

with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    policy_path = config["policy_path"].replace("{H12_ROOT}", h12_root)
    xml_path = config["xml_path"].replace("{H12_ROOT}", h12_root)

    simulation_duration = config["simulation_duration"]
    simulation_dt = config["simulation_dt"]
    control_decimation = config["control_decimation"]

    kps = np.array(config["kps"], dtype=np.float32) * 1.1
    kds = np.array(config["kds"], dtype=np.float32) 

    legs_motor_pos_lower_limit_list = np.array(config["legs_motor_pos_lower_limit_list"], dtype=np.float32)
    legs_motor_pos_upper_limit_list = np.array(config["legs_motor_pos_upper_limit_list"], dtype=np.float32)

    default_angles = np.array(config["default_angles"], dtype=np.float32)

    ang_vel_scale = config["ang_vel_scale"]
    dof_pos_scale = config["dof_pos_scale"]
    dof_vel_scale = config["dof_vel_scale"]
    action_scale = config["action_scale"]
    cmd_scale = np.array(config["cmd_scale"], dtype=np.float32)

    cmd = np.array(config["cmd_init"], dtype=np.float32)

print(f"\nLoaded configuration:")
print(f"  Policy: {policy_path}")
print(f"  XML: {xml_path}")

# Define for 12 DOF lower body only
num_actions = 12  # Only leg DOF
num_obs = 3 + 3 + 3 + 12 + 12 + 12 # base_ang_vel + projected_gravity + velocity_commands + joint_pos_rel + joint_vel_rel + last_action

print(f"  num_actions: {num_actions}, num_obs: {num_obs}")

# define context variables
action = np.zeros(num_actions, dtype=np.float32)
target_dof_pos = default_angles.copy()
obs = np.zeros(num_obs, dtype=np.float32)

counter = 0

# Observation history buffer - 12 DOF only
history_length = 1
obs_dim = 3 + 3 + 3 + 12 + 12 + 12  # base_ang_vel + projected_gravity + velocity_commands + joint_pos_rel(12) + joint_vel_rel(12) + last_action(12)
obs_single = np.zeros(obs_dim, dtype=np.float32)
obs_history = np.zeros((history_length, obs_dim), dtype=np.float32)
last_action = np.zeros(num_actions, dtype=np.float32)

# Load robot model
print(f"\nLoading MuJoCo model from: {xml_path}")
m = mujoco.MjModel.from_xml_path(xml_path)
d = mujoco.MjData(m)
m.opt.timestep = simulation_dt

# load policy
print(f"Loading policy from: {policy_path}")
policy = torch.jit.load(policy_path)

print("\nSetup complete!")

Loading config from: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma/mujoco/h12.yaml
H12 root: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma

Loaded configuration:
  Policy: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma/logs/rsl_rl/h12_locomotion_rma/2026-01-13_12-10-35/exported/policy.pt
  XML: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma/mujoco/h1_2/scene.xml
  num_actions: 12, num_obs: 45

Loading MuJoCo model from: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma/mujoco/h1_2/scene.xml
Loading policy from: /home/niraj/isaac_projects/h12_locomotion_rma/h12_locomotion_rma/logs/rsl_rl/h12_locomotion_rma/2026-01-13_12-10-35/exported/policy.pt

Setup complete!


In [ ]:
# Joint mapping: Isaac Lab to MuJoCo indices
# Left and right joints are interleaved in MuJoCo: left gets 0-5, right gets 6-11
isaac_to_mujoco_idx = np.array([0, 6, 1, 7, 2, 8, 3, 9, 4, 10, 5, 11], dtype=np.int32)
mujoco_to_isaac_idx = np.zeros(num_actions, dtype=np.int32)
for isaac_idx in range(num_actions):
    mjc_idx = isaac_to_mujoco_idx[isaac_idx]
    mujoco_to_isaac_idx[mjc_idx] = isaac_idx
print(f"mujoco_to_isaac_idx: {mujoco_to_isaac_idx}")
print(f"isaac_to_mujoco_idx: {isaac_to_mujoco_idx}")

# Keep default_angles in MuJoCo order (URDF order) for direct indexing
print(f"default_angles (in MuJoCo order): {default_angles}")

# Keep limits in MuJoCo order since we control in MuJoCo space
print(f"Joint limits (in MuJoCo order)")
print(f"  Lower: {legs_motor_pos_lower_limit_list}")
print(f"  Upper: {legs_motor_pos_upper_limit_list}")

mujoco_to_isaac_idx: [ 0  2  4  6  8 10  1  3  5  7  9 11]
isaac_to_mujoco_idx: [ 0  6  1  7  2  8  3  9  4 10  5 11]
default_angles (in MuJoCo order): [ 0.   -0.16  0.    0.36 -0.15  0.    0.   -0.16  0.    0.36 -0.15  0.  ]
target_dof_pos (in Isaac Lab order): [ 0.    0.   -0.16 -0.16  0.    0.    0.36  0.36 -0.15 -0.15  0.    0.  ]
Joint limits (now in Isaac Lab order)
  Lower: [-0.43     -0.43     -3.14     -3.14     -0.43     -3.14     -0.26
 -0.26     -0.897334 -0.897334 -0.261799 -0.261799]
  Upper: [0.43     0.43     2.5      2.5      3.14     0.43     2.05     2.05
 0.523598 0.523598 0.261799 0.261799]


In [66]:
with mujoco.viewer.launch_passive(m, d) as viewer:
    # Close the viewer automatically after simulation_duration wall-seconds.
    start = time.time()
    step_count = 0

    target_dof_pos_mujoco = default_angles.copy()  # Target in MuJoCo order

    while viewer.is_running() and time.time() - start < simulation_duration:
        step_start = time.time()
        
        # Get joint positions and velocities in MuJoCo order
        mjc_qpos = d.qpos[7:]  # All joint positions (MuJoCo order)
        mjc_qvel = d.qvel[6:]  # All joint velocities (MuJoCo order)
        
        # Compute tau in MuJoCo space with MuJoCo-ordered data and gains
        tau_mujoco = pd_control(target_dof_pos_mujoco, mjc_qpos, kps, np.zeros_like(kds), mjc_qvel, kds)
        
        # Apply directly to simulation
        d.ctrl[:] = tau_mujoco
        
        mujoco.mj_step(m, d)

        counter += 1
        step_count += 1
        
        if counter % control_decimation == 0:
                # Get joint data in MuJoCo order, then reorder to Isaac Lab
                mjc_qpos = d.qpos[7:]
                mjc_qvel = d.qvel[6:]
                
                # Reorder to Isaac Lab frame for observations
                qj = (mjc_qpos - default_angles)[isaac_to_mujoco_idx]
                dqj = mjc_qvel[isaac_to_mujoco_idx]
                
                quat = d.qpos[3:7]
                omega = d.qvel[3:6]

                # Normalize observations (in Isaac Lab order)
                qj = qj * dof_pos_scale
                dqj = dqj * dof_vel_scale
                gravity_orientation = get_gravity_orientation(quat)
                omega = omega * ang_vel_scale

                # Build observation vector (in Isaac Lab order)
                obs[:3] = omega
                obs[3:6] = gravity_orientation
                obs[6:9] = cmd * cmd_scale
                obs[9 : 9 + num_actions] = qj
                obs[9 + num_actions : 9 + 2 * num_actions] = dqj
                obs[9 + 2 * num_actions : 9 + 3 * num_actions] = action
                
                obs_tensor = torch.from_numpy(obs).unsqueeze(0)
                
                # Policy inference returns action in Isaac Lab order
                action = policy(obs_tensor).detach().numpy().squeeze()
                
                # Convert Isaac Lab action to MuJoCo order for target positions
                target_dof_pos_isaac = action * action_scale
                target_dof_pos_mujoco = target_dof_pos_isaac[isaac_to_mujoco_idx] + default_angles
                target_dof_pos_mujoco = np.clip(target_dof_pos_mujoco, legs_motor_pos_lower_limit_list, legs_motor_pos_upper_limit_list)

        # Pick up changes to the physics state, apply perturbations, update options from GUI.
        viewer.sync()

        # Rudimentary time keeping, will drift relative to wall clock.
        time_until_next_step = m.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)
